In [6]:
using Random, Distributions,  XLSX, DataFrames, Plots,  CSV, QuasiMonteCarlo, HypothesisTests, HTTP

This function receives the charges of a cluster, a vector of the real cluster of the data, the dict of samples, the cluster that you want to analysis, the version of the cluster classification, and the size os the samples:
- **v0 - algorithm based**
- **v1 - smoker,**
- **v2 - smoker, BMI, age**

The idea is to return plots comparing the samples distributions and the real distribution.

In [2]:
function compare_dist(y::Vector{Float64}, real_cluster::Vector{Float64}, sample::Dict, cluster::Union{Float64,String}, version::String, n::Vector{Int64})

    #criando a distribuicao acumulada de ylabel
    ordered_y = sort(y)
    Fy = zeros(length(ordered_y))
    for i in 1:length(y) 
        Fy[i] = i/length(y)
    end

    if cluster != "base"
        prop_real = sum(real_cluster .== cluster) / length(real_cluster)
    end

    mc_p = plot(ordered_y, Fy, label = "data", legend = :topleft, colour = "black", linewidth = 1.5)
           title!("Monte Carlo Sampling")
    qmc_p = plot(ordered_y, Fy,label = "data", legend = :topleft, colour = "black", linewidth = 1.5)
            title!("QMC LatinHypercubeSample")
    j = 1

    mc_pvalue = zeros(length(n))
    qmc_pvalue = zeros(length(n))

    mc_prop = zeros(length(n))
    qmc_prop = zeros(length(n))

    for i in n

        if cluster != "base"
            mc_y = sort(sample["MC"]["value"][i][findall(j -> j == cluster,sample["MC"]["cluster"][version][i])])
            qmc_y = sort(sample["QMC"]["value"][i][findall(j -> j == cluster,sample["QMC"]["cluster"][version][i])])

            mc_pvalue[j] = pvalue(ApproximateTwoSampleKSTest(y, sample["MC"]["value"][i][findall(j -> j == cluster,sample["MC"]["cluster"][version][i])]))
            qmc_pvalue[j] = pvalue(ApproximateTwoSampleKSTest(y, sample["QMC"]["value"][i][findall(j -> j == cluster,sample["QMC"]["cluster"][version][i])]))
        else
            mc_y = sort(sample["MC"]["value"][i])
            qmc_y = sort(sample["QMC"]["value"][i])

            mc_pvalue[j] = pvalue(ApproximateTwoSampleKSTest(y, sample["MC"]["value"][i]))
            qmc_pvalue[j] = pvalue(ApproximateTwoSampleKSTest(y, sample["QMC"]["value"][i]))
        end

        mc_fy = zeros(length(mc_y))
        qmc_fy = zeros(length(qmc_y))

        for j in 1:length(mc_y)
            mc_fy[j] = j/length(mc_y)
        end

        for j in 1:length(qmc_y)
            qmc_fy[j] = j/length(qmc_y)
        end

        mc_p = plot!(mc_p, mc_y, mc_fy, label = "n = $i", legend = false)
        qmc_p = plot!(qmc_p, qmc_y, qmc_fy,label = "n = $i", legend = false)
       
        if cluster != "base"
            mc_prop[j] = sum(sample["MC"]["cluster"][version][i] .== cluster) / i
            qmc_prop[j] = sum(sample["QMC"]["cluster"][version][i] .== cluster) / i
        end

        j += 1
    end

    pvalue_p = plot(n, mc_pvalue, label = "MC")
               plot!(n, qmc_pvalue, label = "QMC", legend = :bottomright)
               title!("KS Test: P-value")

    if cluster != "base"
        prop_p = plot(n, mc_prop, label = "MC")
                plot!(n, qmc_prop, label = "QMC", legend = :bottomright)
                hline!(n, [prop_real], label = "Real Prop.", colour = "black")
                title!("Proportion convergence")
        
        return mc_p, qmc_p, pvalue_p, prop_p
    else
        return mc_p, qmc_p, pvalue_p, nothing
    end
end

compare_dist (generic function with 1 method)

In [3]:
data_url = "https://github.com/MichaelBarmann/ParamEst_MRE/blob/main/sample_data.csv"
my_file = CSV.File(HTTP.get(data_url).body; header = 1)

base_v1 = XLSX.readdata("Databases/Database.xlsx", "Sheet1!B2:L1339")
base_v2 = CSV.read("Databases/contracts.csv", DataFrame)
dados = Matrix{Float64}(base_v1[:, [7, 10]])

LoadError: AssertionError: File Databases/Database.xlsx not found.